## Все должно исправно работать с версией tensorflow 1.8.0. Если что-то не так, проверяйте версию.

In [20]:
import collections
import os
import sys
import numpy as np
import time
from functools import reduce
from keras.preprocessing import sequence

import tensorflow as tf

In [21]:
tf.__version__

'1.8.0'

# Data processing

Из файла с именами динозавров считываем все, разбиваем все имена на символы, и конкатенируем. Получаем большой массив символов на выходе.

In [22]:
def _read_chars(filename):
    with open(filename, "r") as f:
        names = f.read().lower().split()
        split_names = list(map(lambda x: list(x), names))
        chars = reduce(lambda x, y: x + y, split_names)
        return chars

Считываем имена в один список строк. Затем каждое имя разбиваем на символы и перед ними вставляем токен начала последовательности - "<start\>"

In [23]:
def _read_names(filename):
    with open(filename, "r") as f:
        names = f.read().lower().split()
        split_names = list(map(lambda x: ['<start>'] + list(x), names))
        return split_names

По списку символов строим словарь char_to_id, с помощью которого можно по символу получить его индекс, и словарь id_to_char, с помощью которого можно по индексу получить символ.

In [24]:
def _build_vocab(filename):
    data = _read_chars(filename)
    
    special_tokens = ['<start>', '<eos>', '<pad>']
    
    data += special_tokens
    
    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    chars, _ = list(zip(*count_pairs))
    char_to_id = dict(zip(chars, range(len(chars))))
    id_to_char = {v: k for k, v in char_to_id.items()}

    return char_to_id, id_to_char

Функция _names_to_char_ids - то что получилось на выходе _read_names, переводим в индексы.
```
Пример:
Выход функции _read_names.
['<start>', 'm', 'e', 'l', 'a', 'n', 'o', 'r', 'o', 's', 'a', 'u', 'r', 'u', 's']
['<start>', 'l', 'u', 's', 'i', 't', 'a', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's']
['<start>', 'j', 'i', 'a', 'n', 'g', 'j', 'u', 'n', 'm', 'i', 'a', 'o', 's', 'a', 'u', 'r', 'u', 's']
['<start>', 'a', 's', 't', 'r', 'o', 'd', 'o', 'n']
['<start>', 'o', 'u', 'r', 'a', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's']

Выход функции _names_to_char_ids.
[[28, 14, 7, 9, 0, 5, 4, 3, 4, 1, 0, 2, 3, 2, 1],
 [28, 9, 2, 1, 6, 8, 0, 5, 4, 1, 0, 2, 3, 2, 1],
 [28, 22, 6, 0, 5, 13, 22, 2, 5, 14, 6, 0, 4, 1, 0, 2, 3, 2, 1],
 [28, 0, 1, 8, 3, 4, 15, 4, 5],
 [28, 4, 2, 3, 0, 5, 4, 1, 0, 2, 3, 2, 1]]

```

In [25]:
def _names_to_char_ids(filename, char_to_id):
    data = _read_names(filename)
    
    def name_processing(name):
        return [char_to_id[char] for char in name]
    
    return list(map(name_processing, data))

Функция ptb_raw_data возвращает данные для обучения, подбора параметров и итоговой оценки модели, а также словари char_to_id, id_to_char

In [26]:
def ptb_raw_data(data_path=None):

    train_path = os.path.join(data_path, "dino.train.txt")
    dev_path = os.path.join(data_path, "dino.valid.txt")
    test_path = os.path.join(data_path, "dino.test.txt")

    char_to_id, id_to_char = _build_vocab(train_path)
    train_data = _names_to_char_ids(train_path, char_to_id)
    dev_data = _names_to_char_ids(dev_path, char_to_id)
    test_data = _names_to_char_ids(test_path, char_to_id)
    vocabulary = len(char_to_id)
    print('Vocab len', vocabulary)
    return train_data, dev_data, test_data, char_to_id, id_to_char

Функция _batch_generator разбивает данные на батчи. Возращает вход и выход языковой модели для каждого батча. Выходы это те же входы только смещенные на 1, и с токеном < eos > в конце. 
```
Пример:
X = [[<start>, 1, 2, 3],
     [<start>, 1, 2, 3, 4],
     [<start>, 1, 2]]
Y = [[1, 2, 3, <eos>],
     [1, 2, 3, 4, <eos>],
     [1, 2, <eos>]]
```

Далее каждая строка дополняется токеном < pad > до длины максимального имени.

In [27]:
def _batch_generator(data, batch_size, char_to_id, num_steps):
    n = len(data)
    X = data
    Y = list(map(lambda x: x[1:] + [char_to_id['<eos>']], X))

    batch_padded_X = sequence.pad_sequences(X, 
                                            maxlen=num_steps,
                                            padding='post', 
                                            truncating='post', 
                                            value=char_to_id['<pad>'])

    batch_padded_Y = sequence.pad_sequences(Y, 
                                            maxlen=num_steps,
                                            padding='post', 
                                            truncating='post', 
                                            value=char_to_id['<pad>'])
    
    for k in range(0, n - n % batch_size, batch_size):
        x = np.asarray(batch_padded_X[k: min(k+batch_size, n)])
        y = np.asarray(batch_padded_Y[k: min(k+batch_size, n)])
        yield x, y

Класс ModelInput нужен для того, чтобы каждая из моделей (для train, dev и test) имела свой объект для работы с данными.

In [28]:
class ModelInput(object):
    def __init__(self, config, data, char_to_id, id_to_char, name=None):
        self.name = name
        self.data = data
        self.char_to_id = char_to_id
        self.id_to_char = id_to_char
        self.num_steps = config['num_steps']
        self.batch_size = batch_size = config['batch_size']
        self.epoch_size = len(data) // batch_size
        
    def batch_generator(self):
        return _batch_generator(self.data, 
                               self.batch_size, 
                               self.char_to_id,
                               self.num_steps)


# Model

## Создание плейсхолдеров
```
Создадим ячейки для наших входных данных:
1) `lstm_inputs` - ячейка для входных индексов символов размера (batch_size, num_steps)
2) `lstm_targets` - ячейка в которую помещаются символы из `lstm_inputs` сдвинутые на один влево (то есть предсказания слова на следующем таймстепе)
3) `lr` - ячейка для значения learning rate
```

In [29]:
def build_placeholders(network, config):

    network['lstm_inputs'] = tf.placeholder(shape=(config['batch_size'], 
                                                   config['num_steps']), 
                                            dtype=tf.int32,
                                            name="lstm_inputs")

    network['lstm_targets'] = tf.placeholder(shape=(config['batch_size'], 
                                                    config['num_steps']), 
                                             dtype=tf.int32, 
                                             name="lstm_targets")

    network['lr'] = tf.placeholder(shape=(), dtype=tf.float32, name="lr")
    
    non_pad_ids = tf.not_equal(network['lstm_inputs'], config['pad_id'])
    network['seq_lens'] = tf.reduce_sum(tf.cast(non_pad_ids, tf.int32), axis=1)


## Создание слоя векторных представлений слов  
Слой, преобразующий индексы символов в векторные представления этих символов.

In [30]:
def build_embedding_layer(network, config, is_training):
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", 
                                    shape=[config['vocab_size'], 
                                           config['emb_size']], 
                                    dtype=tf.float32)
        network['inputs'] = tf.nn.embedding_lookup(embedding, network['lstm_inputs'])

    if is_training and config['keep_prob'] < 1:
        network['inputs'] = tf.nn.dropout(network['inputs'], 
                                          config['keep_prob'])

## Создание LSTM ячейки

In [31]:
def make_cell(config, is_training):
    cell = tf.contrib.rnn.LSTMBlockCell(config['hidden_size'], 
                                        forget_bias=0.0)
    if is_training and config['keep_prob'] < 1:
        cell = tf.contrib.rnn.DropoutWrapper(cell, 
                                             output_keep_prob=config['keep_prob'])
    return cell

## Развёртывание LSTM последовательности

На вход функции dynamic_rnn подается нулевое начальное состояние, LSTM ячейка, входы рекуррентного слоя, и длина каждой последовательности в батче, чтобы не обрабатывать паддинги. В результате получаются выходы с каждого таймстепа, и финальные состояния h и c.

In [32]:
def build_rnn_graph(network, config, is_training):
    network['cell'] = make_cell(config, is_training)

    initial_state = network['cell'].zero_state(config['batch_size'], 
                                               dtype=tf.float32)
    network['initial_state'] = initial_state

    outputs, final_state = tf.nn.dynamic_rnn(inputs=network['inputs'], 
                                             cell=network['cell'], 
                                             initial_state=network['initial_state'],
                                             sequence_length=network['seq_lens'])

    network['rnn_outputs'] = outputs
    network['rnn_final_state'] = final_state

## Создание функции потерь

Для вычисления функции потерь требуется маска - матрица размером (batch_size, num_steps), в каждой ячейке которой стоит 0 или 1, обозначающий присутствие или отсутствие паддинга в этой позиции. network['softmax'] здесь вычисляется для того, чтобы генерировать имена, после обучения модели.

In [33]:
def build_loss_function(network, config):
    logits = tf.contrib.layers.fully_connected(network['rnn_outputs'], 
                                               num_outputs=config['vocab_size'],
                                               activation_fn=None)

    network['softmax'] = tf.nn.softmax(logits, axis=2)

    sequence_mask = tf.sequence_mask(network['seq_lens'], 
                                     maxlen=config['num_steps'], 
                                     dtype=tf.float32)
    
    losses = tf.contrib.seq2seq.sequence_loss(logits,
                                              network['lstm_targets'],
                                              weights=sequence_mask,
                                              average_across_timesteps=False,
                                              average_across_batch=False)

    total_words = tf.cast(tf.reduce_sum(network['seq_lens']), tf.float32)

    network['loss'] = tf.reduce_sum(losses) / total_words


## Создание функции обучения
Также в этой функции происходит обрезание градиента по значению 'max_grad_norm'.

In [34]:
def build_train_ops(network, config):
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(network['loss'], tvars), 
                                      config['max_grad_norm'])

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=network['lr'])
#     optimizer = tf.train.AdamOptimizer(learning_rate=network['lr'])

    network['train_op'] = optimizer.apply_gradients(
        zip(grads, tvars), 
        global_step=tf.train.get_or_create_global_step())


## Создание графа

In [35]:
def build_graph(config, input_, is_training):
    network = {'input': input_}

    build_placeholders(network, config)

    build_embedding_layer(network, config, is_training)

    build_rnn_graph(network, config, is_training)

    build_loss_function(network, config)

    if is_training:
        build_train_ops(network, config)

    print("Trainable variables:")
    for var in tf.trainable_variables():
        print(var.name, ':',  var.shape)
    print()

    return network

## Функция для прохождения одной эпохи обучения
Здесь, с помощью функции batch_generator итерируемся по каждому батчу. Модель принимает входы, выходы, learning rate и нулевое начальное состояние. Функция run_epoch возвращает перплексию.

In [17]:
def run_epoch(session, lr, config, model, eval_op=None, verbose=False):
    start_time = time.time()
    loss = 0.0
    iters = 0

    fetches = {
            "loss": model['loss'],
            "rnn_final_state": model['rnn_final_state']
    }

    if eval_op is not None:
        fetches["eval_op"] = eval_op

    def make_zero_state():
        state = tf.nn.rnn_cell.LSTMStateTuple(
            c = np.zeros((config['batch_size'], config['hidden_size'])), 
            h = np.zeros((config['batch_size'], config['hidden_size']))
        )

        return state

    initial_state = make_zero_state()
    for step, (X, Y) in enumerate(model['input'].batch_generator()):

        feed_dict = {
                     model['lstm_inputs']: X, 
                     model['lstm_targets']: Y, 
                     model['initial_state']: initial_state,
                     model['lr']: lr
                    }

        vals = session.run(fetches, feed_dict)
        local_loss = vals["loss"]
#         initial_state = vals["rnn_final_state"]

        loss += local_loss
        iters += 1

    return np.exp(loss / iters)

## Функция для сэмплирования имен

In [40]:
def sample(session, config, model):
    fetches = {
            "rnn_final_state": model['rnn_final_state'],
            "softmax": model['softmax']
    }

    
    def make_zero_state():
        state = tf.nn.rnn_cell.LSTMStateTuple(
            c = np.zeros((config['batch_size'], config['hidden_size'])), 
            h = np.zeros((config['batch_size'], config['hidden_size']))
        )

        return state
    
    initial_state = make_zero_state()
    
    start_id = model['input'].char_to_id['<start>']
    
    idx = np.ones((config['batch_size'], config['num_steps']), dtype=np.int32) * start_id
    
    result = []
    for i in range(config['num_steps']):
        fetches['softmax'] = model['softmax']
        feed_dict = {
                     model['lstm_inputs']: np.array(idx),
                     model['initial_state']: initial_state,
                    }

        vals = session.run(fetches, feed_dict)
        initial_state = vals["rnn_final_state"]
        
        next_words = np.argmax(vals['softmax'], axis=2)[:, i]
#         if i == 0:
#             next_words[:] = model['input'].char_to_id['s']
        
        if i < config['num_steps'] - 1:
            idx[:, i+1] = next_words

        result.append([model['input'].id_to_char[x] for x in next_words])

    result = list(zip(*result))
    result = [''.join(r) for r in result][0]
    result = result.split('<eos>')[0]
    return result

In [41]:
def get_small_config():
    config = {'lr': 1.0, 'lr_decay': 0.5,
              'max_grad_norm': 5, 'emb_size': 200,
              'hidden_size': 200, 'keep_prob': 1.0, 
              'max_epoch': 4, 'max_max_epoch': 13, 
              'batch_size': 20, 'vocab_size': 30,
              'rnn_type': 'dynamic_rnn'}
    return config

def get_small_eval_config():
    eval_config = get_small_config()
#     eval_config['batch_size'] = 1
#     eval_config['num_steps'] = 1
    return eval_config

def main():

    config = get_small_config()

    data_path = 'DINO/'

#     save_path = None
#     restore_path = 'models/epoch_13'
    save_path = 'models/'
    restore_path = None
    
    decayed_lr = config['lr']

    if  save_path is not None and not os.path.exists(save_path):
        os.makedirs(save_path)

    raw_data = ptb_raw_data(data_path)
    train_data, dev_data, test_data, char_to_id, id_to_char = raw_data

    max_len = max(map(len, train_data + dev_data + test_data))
    config['num_steps'] = max_len + 1

    config['pad_id'] = char_to_id['<pad>']

    with tf.Graph().as_default():
        initializer = tf.random_uniform_initializer(-0.1, 0.1)
        
        with tf.name_scope("Train"):
            train_input = ModelInput(config=config, 
                                   data=train_data, 
                                   char_to_id=char_to_id, 
                                   id_to_char=id_to_char, 
                                   name="TrainInput")

            with tf.variable_scope("Model", reuse=None, initializer=initializer):
                train_model = build_graph(config, input_=train_input, is_training=True)

        with tf.name_scope("Dev"):
            dev_input = ModelInput(config=config, 
                                 data=dev_data,
                                 char_to_id=char_to_id, 
                                 id_to_char=id_to_char, 
                                 name="DevInput")

            with tf.variable_scope("Model", reuse=True, initializer=initializer):
                dev_model = build_graph(config, input_=dev_input, is_training=False)

        with tf.name_scope("Test"):
            test_input = ModelInput(config=config, 
                                  data=test_data,
                                  char_to_id=char_to_id, 
                                  id_to_char=id_to_char,
                                  name="TestInput")

            with tf.variable_scope("Model", reuse=True, initializer=initializer):
                test_model = build_graph(config, input_=test_input, is_training=False)

        with tf.Session() as sess:
            saver = tf.train.Saver()
            sess.run(tf.global_variables_initializer())
            
            # Если хотим загрузить обученную модель, то restore_path должен быть не None, пример сверху
            if restore_path is not None:
                saver = tf.train.import_meta_graph(restore_path + '.meta')
                saver.restore(sess, tf.train.latest_checkpoint('models/'))
            
            # Цикл до максимального числа эпох, после каждой эпохи обучения, замеряем перплексию на dev
            for i in range(config['max_max_epoch']):
                lr_decay = config['lr_decay'] ** max(i + 1 - config['max_epoch'], 0.0)
                decayed_lr = config['lr'] * lr_decay

                print("Epoch: %d Learning rate: %.3f" % (i + 1, decayed_lr))
                train_perplexity = run_epoch(sess, 
                                             decayed_lr, 
                                             config, 
                                             train_model, 
                                             eval_op=train_model['train_op'], 
                                             verbose=True)
                print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))

                dev_perplexity = run_epoch(sess, decayed_lr, config, dev_model)
                print("Epoch: %d Dev Perplexity: %.3f" % (i + 1, dev_perplexity))

                name = sample(sess, config, test_model)
                print(name)

            # Итоговая оценка обученной модели
            test_perplexity = run_epoch(sess, 
                                        decayed_lr, 
                                        config, 
                                        test_model)
    
            print("Test Perplexity: %.3f" % test_perplexity)
            
            # Сохраняем обученную модель
            if save_path is not None:
                filename = os.path.join(save_path, 'epoch_{}'.format(config['max_max_epoch']))
                saver.save(sess, filename)

main()

Vocab len 29
Trainable variables:
Model/embedding:0 : (30, 200)
Model/rnn/lstm_cell/kernel:0 : (400, 800)
Model/rnn/lstm_cell/bias:0 : (800,)
Model/fully_connected/weights:0 : (200, 30)
Model/fully_connected/biases:0 : (30,)

Trainable variables:
Model/embedding:0 : (30, 200)
Model/rnn/lstm_cell/kernel:0 : (400, 800)
Model/rnn/lstm_cell/bias:0 : (800,)
Model/fully_connected/weights:0 : (200, 30)
Model/fully_connected/biases:0 : (30,)

Trainable variables:
Model/embedding:0 : (30, 200)
Model/rnn/lstm_cell/kernel:0 : (400, 800)
Model/rnn/lstm_cell/bias:0 : (800,)
Model/fully_connected/weights:0 : (200, 30)
Model/fully_connected/biases:0 : (30,)

Epoch: 1 Learning rate: 1.000
Epoch: 1 Train Perplexity: 16.792
Epoch: 1 Dev Perplexity: 12.910
aus
Epoch: 2 Learning rate: 1.000
Epoch: 2 Train Perplexity: 11.330
Epoch: 2 Dev Perplexity: 9.190
aus
Epoch: 3 Learning rate: 1.000
Epoch: 3 Train Perplexity: 8.767
Epoch: 3 Dev Perplexity: 7.851
aurus
Epoch: 4 Learning rate: 1.000
Epoch: 4 Train Perp